In [1]:
from utils import *

In [2]:
vtestPath = './input/opencv_data/vtest.avi'
import os
assert os.path.exists(vtestPath)

In [3]:
capture = cv.VideoCapture(vtestPath)
assert capture.isOpened(), 'video %s could not be opened'%vtestPath
ret, frame = capture.read()


In [4]:
from CaptureUtils import *
c = CaptureUtils(frame, mode=1, color=(0, 255, 255))
rect = c.startCapture()[0]

In [5]:
rect = np.array(rect)
x, y = rect[0]
w, h = rect[1]-rect[0]
x, y, w, h

(498, 155, 32, 81)

In [6]:
# 设置初始ROI来追踪
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
roi_hist = cv.normalize(roi_hist,0,255,cv.NORM_MINMAX)

In [7]:
track_window = (x, y, w, h)
# 设置终止条件，可以是10次迭代，也可以至少移动1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = capture.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # 应用meanshift来获取新位置
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        # 在图像上绘制
        x,y,w,h = track_window
        dest = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('',dest)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
capture.release()

In [8]:
capture = cv.VideoCapture(vtestPath)
assert capture.isOpened(), 'video %s could not be opened'%vtestPath
ret, frame = capture.read()
x, y = rect[0]
w, h = rect[1]-rect[0]
x, y, w, h
track_window = (x, y, w, h)
# 设置追踪的ROI窗口
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
# 设置终止条件，可以是10次迭代，有可以至少移动1个像素
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = capture.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # 应用camshift 到新位置
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # # 在图像上画出来
        # pts = cv.boxPoints(ret)
        # pts = np.int0(pts)
        # dest = cv.polylines(frame,[pts],True, 255,2)
        ret_np = np.array(ret[:2]).astype(np.int32)
        dest = cv.ellipse(frame, ret_np[0], ret_np[1], ret[2], 0, 360, (0, 255, 0))
        cv.imshow('',dest)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
capture.release()